# Módulo 5: Tarefas Fundamentais em Visão Computacional## Objetivos de Aprendizagem- Dominar as principais tarefas de visão computacional- Compreender diferenças entre classificação, detecção e segmentação- Conhecer arquiteturas específicas para cada tarefa- Implementar soluções práticas com PyTorch- Analisar métricas de avaliação específicas---## 5.1 Classificação de Imagens**Classificação de Imagens** é a tarefa de atribuir uma ou mais labels (etiquetas) a uma imagem completa, determinando a categoria ou classe à qual ela pertence.![Classificação de Imagens](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/classificacao_imagens.png)### Definição e Características**Definição:**- **Entrada**: Uma imagem completa- **Saída**: Uma ou mais classes/categorias- **Objetivo**: Determinar "o que" está na imagem- **Granularidade**: Nível de imagem inteira**Características Principais:**- **Uma imagem = Uma classe**: Cada imagem recebe uma label principal- **Classes mutuamente exclusivas**: Geralmente uma classe por imagem- **Classificação multi-label**: Possível em alguns casos- **Hierárquica**: Classes podem ter subclasses### Evolução das Tarefas**Progressão Histórica:**- **2012 - Classificação**: AlexNet revoluciona ImageNet- **2014 - Detecção**: R-CNN introduz detecção baseada em regiões- **2015 - Segmentação**: U-Net para segmentação médica- **2016+**: Evoluções e melhorias incrementais![Evolução das Tarefas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/evolucao_tarefas.png)**Tendências Observadas:**- **Complexidade**: Aumento gradual da granularidade- **Precisão**: Melhoria constante das métricas- **Velocidade**: Otimização para tempo real- **Aplicação**: Expansão para novos domínios### Arquiteturas Específicas**Modelos Clássicos:**- **VGG**: Arquitetura uniforme com kernels 3×3- **ResNet**: Skip connections para redes profundas- **EfficientNet**: Otimização de eficiência computacional- **DenseNet**: Conexões densas entre camadas**Características das Arquiteturas:**- **Camadas convolucionais**: Extração de características- **Pooling**: Redução de dimensionalidade- **Fully connected**: Classificação final- **Softmax**: Probabilidades normalizadas**Referências:**- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)

## 5.2 Demonstração Prática: Classificação de Imagens

Vamos implementar e visualizar diferentes aspectos da classificação:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

class ImageClassificationDemo:
    """Demonstração de classificação de imagens"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_sample_images(self):
        """Cria imagens de exemplo para demonstração"""
        
        # Criar imagens sintéticas representando diferentes classes
        images = {}
        
        # Classe 1: Círculos
        img_circle = np.zeros((224, 224, 3), dtype=np.uint8)
        cv2.circle(img_circle, (112, 112), 50, (255, 0, 0), -1)
        cv2.circle(img_circle, (112, 112), 30, (0, 255, 0), -1)
        images['circle'] = img_circle
        
        # Classe 2: Retângulos
        img_rect = np.zeros((224, 224, 3), dtype=np.uint8)
        cv2.rectangle(img_rect, (50, 50), (174, 174), (0, 0, 255), -1)
        cv2.rectangle(img_rect, (80, 80), (144, 144), (255, 255, 0), -1)
        images['rectangle'] = img_rect
        
        # Classe 3: Triângulos
        img_triangle = np.zeros((224, 224, 3), dtype=np.uint8)
        pts = np.array([[112, 50], [50, 174], [174, 174]], np.int32)
        cv2.fillPoly(img_triangle, [pts], (255, 0, 255))
        images['triangle'] = img_triangle
        
        # Classe 4: Linhas
        img_lines = np.zeros((224, 224, 3), dtype=np.uint8)
        cv2.line(img_lines, (50, 50), (174, 174), (0, 255, 255), 5)
        cv2.line(img_lines, (174, 50), (50, 174), (255, 255, 0), 5)
        images['lines'] = img_lines
        
        # Classe 5: Ruído
        img_noise = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
        images['noise'] = img_noise
        
        return images
    
    def create_simple_classifier(self, num_classes=5):
        """Cria um classificador simples"""
        
        class SimpleClassifier(nn.Module):
            def __init__(self, num_classes):
                super(SimpleClassifier, self).__init__()
                
                # Camadas convolucionais
                self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2)
                self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2)
                self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
                
                # Pooling
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
                
                # Fully connected
                self.fc1 = nn.Linear(128 * 3 * 3, 256)
                self.fc2 = nn.Linear(256, num_classes)
                
                # Dropout
                self.dropout = nn.Dropout(0.5)
                
            def forward(self, x):
                # Camadas convolucionais
                x = F.relu(self.conv1(x))
                x = self.pool(x)
                x = F.relu(self.conv2(x))
                x = self.pool(x)
                x = F.relu(self.conv3(x))
                x = self.pool(x)
                
                # Flatten
                x = x.view(x.size(0), -1)
                
                # Fully connected
                x = F.relu(self.fc1(x))
                x = self.dropout(x)
                x = self.fc2(x)
                
                return x
        
        return SimpleClassifier(num_classes)
    
    def simulate_classification(self, images, model):
        """Simula classificação das imagens"""
        
        model.eval()
        results = {}
        
        # Classes simuladas
        class_names = ['Círculo', 'Retângulo', 'Triângulo', 'Linhas', 'Ruído']
        
        with torch.no_grad():
            for name, img in images.items():
                # Converter para tensor
                img_tensor = torch.FloatTensor(img).permute(2, 0, 1).unsqueeze(0) / 255.0
                
                # Fazer predição
                output = model(img_tensor)
                probabilities = F.softmax(output, dim=1)
                
                # Simular predição realista
                if name == 'circle':
                    pred_class = 0
                    confidence = 0.95
                elif name == 'rectangle':
                    pred_class = 1
                    confidence = 0.92
                elif name == 'triangle':
                    pred_class = 2
                    confidence = 0.88
                elif name == 'lines':
                    pred_class = 3
                    confidence = 0.85
                else:  # noise
                    pred_class = 4
                    confidence = 0.78
                
                results[name] = {
                    'image': img,
                    'predicted_class': pred_class,
                    'class_name': class_names[pred_class],
                    'confidence': confidence,
                    'probabilities': probabilities.numpy()[0]
                }
        
        return results
    
    def visualize_classification_results(self, results):
        """Visualiza resultados da classificação"""
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        class_names = ['Círculo', 'Retângulo', 'Triângulo', 'Linhas', 'Ruído']
        colors = ['red', 'blue', 'green', 'yellow', 'purple']
        
        # Plotar imagens e resultados
        for i, (name, result) in enumerate(results.items()):
            if i < 5:  # Apenas as primeiras 5 imagens
                row = i // 3
                col = i % 3
                
                # Imagem
                axes[row, col].imshow(result['image'])
                axes[row, col].set_title(f'{result["class_name"]}\nConfiança: {result["confidence"]:.2f}')
                axes[row, col].axis('off')
        
        # Gráfico de probabilidades
        all_probs = np.array([result['probabilities'] for result in results.values()])
        
        x = np.arange(len(class_names))
        width = 0.15
        
        for i, (name, result) in enumerate(results.items()):
            axes[1, 2].bar(x + i*width, result['probabilities'], width, 
                          label=name, color=colors[i], alpha=0.7)
        
        axes[1, 2].set_title('Probabilidades por Classe')
        axes[1, 2].set_xlabel('Classes')
        axes[1, 2].set_ylabel('Probabilidade')
        axes[1, 2].set_xticks(x + width * 2)
        axes[1, 2].set_xticklabels(class_names)
        axes[1, 2].legend()
        axes[1, 2].set_ylim(0, 1)
        
        plt.suptitle('Classificação de Imagens - Resultados', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return results
    
    def analyze_classification_metrics(self, results):
        """Analisa métricas de classificação"""
        
        # Calcular métricas simuladas
        total_images = len(results)
        correct_predictions = sum(1 for r in results.values() if r['confidence'] > 0.8)
        accuracy = correct_predictions / total_images
        
        # Calcular precisão por classe
        class_precision = {}
        class_recall = {}
        
        for i, class_name in enumerate(['Círculo', 'Retângulo', 'Triângulo', 'Linhas', 'Ruído']):
            # Simular métricas realistas
            if i == 0:  # Círculo
                precision = 0.95
                recall = 0.90
            elif i == 1:  # Retângulo
                precision = 0.92
                recall = 0.88
            elif i == 2:  # Triângulo
                precision = 0.88
                recall = 0.85
            elif i == 3:  # Linhas
                precision = 0.85
                recall = 0.82
            else:  # Ruído
                precision = 0.78
                recall = 0.75
            
            class_precision[class_name] = precision
            class_recall[class_name] = recall
        
        # Visualizar métricas
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Gráfico de precisão
        classes = list(class_precision.keys())
        precisions = list(class_precision.values())
        
        axes[0].bar(classes, precisions, color='lightblue')
        axes[0].set_title('Precisão por Classe')
        axes[0].set_ylabel('Precisão')
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(precisions):
            axes[0].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        # Gráfico de recall
        recalls = list(class_recall.values())
        
        axes[1].bar(classes, recalls, color='lightgreen')
        axes[1].set_title('Recall por Classe')
        axes[1].set_ylabel('Recall')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(recalls):
            axes[1].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        # Gráfico de F1-Score
        f1_scores = [2 * (p * r) / (p + r) for p, r in zip(precisions, recalls)]
        
        axes[2].bar(classes, f1_scores, color='lightcoral')
        axes[2].set_title('F1-Score por Classe')
        axes[2].set_ylabel('F1-Score')
        axes[2].tick_params(axis='x', rotation=45)
        axes[2].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(f1_scores):
            axes[2].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        plt.suptitle('Métricas de Classificação', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DAS MÉTRICAS ===")
        print(f"Acurácia Geral: {accuracy:.2f}")
        print(f"Precisão Média: {np.mean(precisions):.2f}")
        print(f"Recall Médio: {np.mean(recalls):.2f}")
        print(f"F1-Score Médio: {np.mean(f1_scores):.2f}")
        
        return {
            'accuracy': accuracy,
            'precision': class_precision,
            'recall': class_recall,
            'f1_score': dict(zip(classes, f1_scores))
        }

def demonstrate_image_classification():
    """Demonstra classificação de imagens"""
    
    print("=== Demonstração de Classificação de Imagens ===")
    
    # Criar instância
    demo = ImageClassificationDemo()
    
    # Criar imagens de exemplo
    print("\nCriando imagens de exemplo...")
    images = demo.create_sample_images()
    
    # Criar modelo
    print("\nCriando modelo de classificação...")
    model = demo.create_simple_classifier(num_classes=5)
    
    # Simular classificação
    print("\nSimulando classificação...")
    results = demo.simulate_classification(images, model)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    demo.visualize_classification_results(results)
    
    # Analisar métricas
    print("\nAnalisando métricas...")
    metrics = demo.analyze_classification_metrics(results)
    
    return results, metrics

# Executar demonstração
classification_results, classification_metrics = demonstrate_image_classification()

### Análise dos Resultados

**Classificação Observada:**

1. **Círculos**: 95% de confiança, alta precisão
2. **Retângulos**: 92% de confiança, boa precisão
3. **Triângulos**: 88% de confiança, precisão moderada
4. **Linhas**: 85% de confiança, precisão moderada
5. **Ruído**: 78% de confiança, menor precisão

**Insights Importantes:**
- **Formas geométricas**: Mais fáceis de classificar
- **Padrões complexos**: Requerem mais dados
- **Ruído**: Desafia a classificação
- **Métricas**: Precisão, recall e F1-score

**Referências:**
- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)


## 5.3 Detecção de Objetos**Detecção de Objetos** é a tarefa de localizar e classificar múltiplos objetos em uma imagem, retornando bounding boxes e classes para cada objeto detectado.![Detecção de Objetos](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/deteccao_objetos.png)### Definição e Características**Definição:**- **Entrada**: Uma imagem completa- **Saída**: Bounding boxes + classes para cada objeto- **Objetivo**: Determinar "o que" e "onde" estão os objetos- **Granularidade**: Nível de objeto individual**Características Principais:**- **Múltiplos objetos**: Uma imagem pode conter vários objetos- **Localização**: Coordenadas (x, y, width, height)- **Classificação**: Classe de cada objeto- **Confiança**: Probabilidade de detecção### Arquiteturas Específicas**Modelos Clássicos:**- **YOLO**: You Only Look Once, detecção em tempo real- **Faster R-CNN**: Region-based CNN com RPN- **SSD**: Single Shot Detector- **RetinaNet**: Focal Loss para classes desbalanceadas![Arquiteturas Detecção](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/arquiteturas_deteccao.png)**Características das Arquiteturas:**- **Two-stage**: R-CNN, Faster R-CNN (mais preciso)- **One-stage**: YOLO, SSD (mais rápido)- **Anchor-based**: Usa anchors para detecção- **Anchor-free**: Detecta sem anchors**Referências:**- [You Only Look Once: Unified, Real-Time Object Detection - Redmon et al.](https://arxiv.org/abs/1506.02640)- [Faster R-CNN: Towards Real-Time Object Detection - Ren et al.](https://arxiv.org/abs/1506.01497)

## 5.4 Demonstração Prática: Detecção de Objetos

Vamos implementar e visualizar detecção de objetos:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.patches import Rectangle

class ObjectDetectionDemo:
    """Demonstração de detecção de objetos"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_sample_scene(self):
        """Cria uma cena de exemplo com múltiplos objetos"""
        
        # Criar imagem base
        img = np.zeros((400, 600, 3), dtype=np.uint8)
        
        # Adicionar fundo
        img[:] = (50, 50, 50)  # Cinza escuro
        
        # Adicionar objetos
        objects = []
        
        # Objeto 1: Círculo vermelho
        cv2.circle(img, (150, 100), 40, (0, 0, 255), -1)
        objects.append({
            'class': 'Círculo',
            'bbox': [110, 60, 80, 80],  # x, y, w, h
            'confidence': 0.95
        })
        
        # Objeto 2: Retângulo azul
        cv2.rectangle(img, (300, 80), (450, 180), (255, 0, 0), -1)
        objects.append({
            'class': 'Retângulo',
            'bbox': [300, 80, 150, 100],
            'confidence': 0.92
        })
        
        # Objeto 3: Triângulo verde
        pts = np.array([[100, 250], [50, 350], [150, 350]], np.int32)
        cv2.fillPoly(img, [pts], (0, 255, 0))
        objects.append({
            'class': 'Triângulo',
            'bbox': [50, 250, 100, 100],
            'confidence': 0.88
        })
        
        # Objeto 4: Linha amarela
        cv2.line(img, (400, 200), (550, 300), (0, 255, 255), 8)
        objects.append({
            'class': 'Linha',
            'bbox': [400, 200, 150, 100],
            'confidence': 0.85
        })
        
        # Objeto 5: Círculo pequeno
        cv2.circle(img, (500, 150), 25, (255, 0, 255), -1)
        objects.append({
            'class': 'Círculo',
            'bbox': [475, 125, 50, 50],
            'confidence': 0.78
        })
        
        return img, objects
    
    def simulate_detection(self, img, objects):
        """Simula detecção de objetos"""
        
        # Simular detecções com algumas variações
        detections = []
        
        for obj in objects:
            # Adicionar pequeno ruído às coordenadas
            noise_x = np.random.randint(-5, 6)
            noise_y = np.random.randint(-5, 6)
            noise_w = np.random.randint(-3, 4)
            noise_h = np.random.randint(-3, 4)
            
            bbox = obj['bbox'].copy()
            bbox[0] += noise_x
            bbox[1] += noise_y
            bbox[2] += noise_w
            bbox[3] += noise_h
            
            # Garantir que as coordenadas estão dentro da imagem
            bbox[0] = max(0, min(bbox[0], img.shape[1] - bbox[2]))
            bbox[1] = max(0, min(bbox[1], img.shape[0] - bbox[3]))
            
            detections.append({
                'class': obj['class'],
                'bbox': bbox,
                'confidence': obj['confidence'] + np.random.normal(0, 0.02)
            })
        
        # Adicionar algumas detecções falsas
        false_detections = [
            {
                'class': 'Círculo',
                'bbox': [200, 300, 60, 60],
                'confidence': 0.45
            },
            {
                'class': 'Retângulo',
                'bbox': [50, 50, 80, 40],
                'confidence': 0.38
            }
        ]
        
        detections.extend(false_detections)
        
        return detections
    
    def visualize_detections(self, img, detections, threshold=0.5):
        """Visualiza detecções de objetos"""
        
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        
        # Imagem original
        axes[0].imshow(img)
        axes[0].set_title('Imagem Original')
        axes[0].axis('off')
        
        # Imagem com detecções
        axes[1].imshow(img)
        
        # Cores para diferentes classes
        class_colors = {
            'Círculo': 'red',
            'Retângulo': 'blue',
            'Triângulo': 'green',
            'Linha': 'yellow'
        }
        
        # Desenhar bounding boxes
        for det in detections:
            if det['confidence'] >= threshold:
                bbox = det['bbox']
                x, y, w, h = bbox
                
                # Cor baseada na classe
                color = class_colors.get(det['class'], 'white')
                
                # Desenhar retângulo
                rect = Rectangle((x, y), w, h, 
                              linewidth=2, 
                              edgecolor=color, 
                              facecolor='none')
                axes[1].add_patch(rect)
                
                # Adicionar label
                axes[1].text(x, y-5, 
                          f'{det["class"]} {det["confidence"]:.2f}',
                          color=color, 
                          fontsize=10, 
                          fontweight='bold',
                          bbox=dict(boxstyle='round,pad=0.3', 
                                   facecolor='white', 
                                   alpha=0.8))
        
        axes[1].set_title(f'Detecções (threshold ≥ {threshold})')
        axes[1].axis('off')
        
        plt.suptitle('Detecção de Objetos', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return detections
    
    def analyze_detection_metrics(self, detections, threshold=0.5):
        """Analisa métricas de detecção"""
        
        # Filtrar detecções por threshold
        filtered_detections = [d for d in detections if d['confidence'] >= threshold]
        
        # Calcular métricas simuladas
        total_objects = 5  # Objetos reais na cena
        detected_objects = len([d for d in filtered_detections if d['confidence'] >= 0.7])
        false_positives = len([d for d in filtered_detections if d['confidence'] < 0.7])
        
        # Métricas
        precision = detected_objects / (detected_objects + false_positives) if (detected_objects + false_positives) > 0 else 0
        recall = detected_objects / total_objects
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        # Visualizar métricas
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Gráfico de confiança
        confidences = [d['confidence'] for d in detections]
        classes = [d['class'] for d in detections]
        
        colors = ['red' if c >= threshold else 'gray' for c in confidences]
        
        axes[0].bar(range(len(confidences)), confidences, color=colors, alpha=0.7)
        axes[0].axhline(y=threshold, color='red', linestyle='--', label=f'Threshold = {threshold}')
        axes[0].set_title('Confiança das Detecções')
        axes[0].set_ylabel('Confiança')
        axes[0].set_xlabel('Detecções')
        axes[0].legend()
        axes[0].set_ylim(0, 1)
        
        # Gráfico de métricas
        metrics = ['Precisão', 'Recall', 'F1-Score']
        values = [precision, recall, f1_score]
        
        axes[1].bar(metrics, values, color=['lightblue', 'lightgreen', 'lightcoral'])
        axes[1].set_title('Métricas de Detecção')
        axes[1].set_ylabel('Valor')
        axes[1].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(values):
            axes[1].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        # Gráfico de distribuição por classe
        class_counts = {}
        for det in filtered_detections:
            class_name = det['class']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        if class_counts:
            classes = list(class_counts.keys())
            counts = list(class_counts.values())
            
            axes[2].bar(classes, counts, color=['red', 'blue', 'green', 'yellow'])
            axes[2].set_title('Detecções por Classe')
            axes[2].set_ylabel('Número de Detecções')
            axes[2].tick_params(axis='x', rotation=45)
        
        plt.suptitle('Análise de Detecção de Objetos', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DAS MÉTRICAS ===")
        print(f"Total de Objetos: {total_objects}")
        print(f"Detecções Acima do Threshold: {len(filtered_detections)}")
        print(f"Precisão: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1-Score: {f1_score:.2f}")
        
        return {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'total_objects': total_objects,
            'detected_objects': detected_objects,
            'false_positives': false_positives
        }

def demonstrate_object_detection():
    """Demonstra detecção de objetos"""
    
    print("=== Demonstração de Detecção de Objetos ===")
    
    # Criar instância
    demo = ObjectDetectionDemo()
    
    # Criar cena de exemplo
    print("\nCriando cena de exemplo...")
    img, objects = demo.create_sample_scene()
    
    # Simular detecção
    print("\nSimulando detecção...")
    detections = demo.simulate_detection(img, objects)
    
    # Visualizar detecções
    print("\nVisualizando detecções...")
    demo.visualize_detections(img, detections, threshold=0.5)
    
    # Analisar métricas
    print("\nAnalisando métricas...")
    metrics = demo.analyze_detection_metrics(detections, threshold=0.5)
    
    return detections, metrics

# Executar demonstração
detection_results, detection_metrics = demonstrate_object_detection()

### Análise dos Resultados

**Detecção Observada:**

1. **Objetos Detectados**: 5 objetos principais
2. **Confiança**: Varia de 0.78 a 0.95
3. **Falsos Positivos**: 2 detecções com baixa confiança
4. **Threshold**: 0.5 para filtrar detecções

**Insights Importantes:**
- **Localização**: Bounding boxes precisos
- **Classificação**: Classes corretas identificadas
- **Confiança**: Variação realista
- **Métricas**: Precisão, recall e F1-score

**Referências:**
- [You Only Look Once: Unified, Real-Time Object Detection - Redmon et al.](https://arxiv.org/abs/1506.02640)


## 5.5 Segmentação de Imagens**Segmentação de Imagens** é a tarefa de dividir uma imagem em regiões ou segmentos, onde cada pixel pertence a uma classe específica.![Segmentação de Imagens](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/segmentacao_imagens.png)### Tipos de Segmentação**1. Segmentação Semântica:**- **Objetivo**: Classificar cada pixel- **Saída**: Mapa de classes por pixel- **Exemplo**: Céu, árvore, carro, pessoa- **Granularidade**: Nível de pixel**2. Segmentação de Instância:**- **Objetivo**: Separar instâncias individuais- **Saída**: Mapa de instâncias por pixel- **Exemplo**: Carro 1, carro 2, pessoa 1- **Granularidade**: Nível de instância**3. Segmentação Panóptica:**- **Objetivo**: Combina semântica + instância- **Saída**: Classes + instâncias- **Exemplo**: Céu, árvore, carro 1, carro 2- **Granularidade**: Híbrida### Arquiteturas Específicas**Modelos Clássicos:**- **U-Net**: Arquitetura em U para segmentação médica- **DeepLab**: Atrous convolutions para contexto- **Mask R-CNN**: Detecção + segmentação de instância- **PSPNet**: Pyramid Scene Parsing![Arquiteturas Segmentação](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/arquiteturas_segmentacao.png)**Características das Arquiteturas:**- **Encoder-Decoder**: Redução + expansão- **Skip connections**: Preservação de detalhes- **Atrous convolutions**: Campo receptivo ampliado- **Multi-scale**: Processamento em múltiplas escalas**Referências:**- [U-Net: Convolutional Networks for Biomedical Image Segmentation - Ronneberger et al.](https://arxiv.org/abs/1505.04597)- [DeepLab: Semantic Image Segmentation - Chen et al.](https://arxiv.org/abs/1606.00915)

## 5.6 Demonstração Prática: Segmentação de Imagens

Vamos implementar e visualizar segmentação de imagens:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.colors import ListedColormap

class ImageSegmentationDemo:
    """Demonstração de segmentação de imagens"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_sample_scene(self):
        """Cria uma cena de exemplo para segmentação"""
        
        # Criar imagem base
        img = np.zeros((300, 400, 3), dtype=np.uint8)
        
        # Adicionar fundo (céu)
        img[:150, :] = (135, 206, 235)  # Azul claro
        
        # Adicionar chão
        img[150:, :] = (34, 139, 34)  # Verde
        
        # Adicionar objetos
        
        # Árvore
        cv2.rectangle(img, (50, 100), (80, 150), (139, 69, 19), -1)  # Tronco
        cv2.circle(img, (65, 100), 30, (0, 100, 0), -1)  # Folhas
        
        # Casa
        cv2.rectangle(img, (200, 120), (300, 200), (139, 69, 19), -1)  # Casa
        cv2.rectangle(img, (220, 140), (280, 200), (255, 255, 255), -1)  # Porta
        cv2.rectangle(img, (240, 160), (260, 180), (255, 255, 0), -1)  # Janela
        
        # Telhado
        pts = np.array([[200, 120], [250, 80], [300, 120]], np.int32)
        cv2.fillPoly(img, [pts], (139, 0, 0))  # Telhado
        
        # Nuvem
        cv2.circle(img, (320, 60), 25, (255, 255, 255), -1)
        cv2.circle(img, (340, 60), 20, (255, 255, 255), -1)
        cv2.circle(img, (300, 60), 20, (255, 255, 255), -1)
        
        return img
    
    def create_ground_truth_mask(self, img):
        """Cria máscara de ground truth para segmentação"""
        
        # Criar máscara baseada na imagem
        mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        
        # Classe 0: Fundo
        mask[:] = 0
        
        # Classe 1: Céu (azul claro)
        sky_mask = np.all(img == [135, 206, 235], axis=2)
        mask[sky_mask] = 1
        
        # Classe 2: Chão (verde)
        ground_mask = np.all(img == [34, 139, 34], axis=2)
        mask[ground_mask] = 2
        
        # Classe 3: Árvore
        tree_mask = np.all(img == [0, 100, 0], axis=2)
        mask[tree_mask] = 3
        
        # Classe 4: Casa
        house_mask = np.all(img == [139, 69, 19], axis=2)
        mask[house_mask] = 4
        
        # Classe 5: Telhado
        roof_mask = np.all(img == [139, 0, 0], axis=2)
        mask[roof_mask] = 5
        
        # Classe 6: Nuvem
        cloud_mask = np.all(img == [255, 255, 255], axis=2)
        mask[cloud_mask] = 6
        
        return mask
    
    def simulate_segmentation(self, img, mask):
        """Simula segmentação da imagem"""
        
        # Simular segmentação com pequenas variações
        predicted_mask = mask.copy()
        
        # Adicionar ruído para simular imperfeições
        noise = np.random.randint(0, 7, mask.shape)
        
        # Aplicar ruído em algumas regiões
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if np.random.random() < 0.05:  # 5% de chance de erro
                    predicted_mask[i, j] = noise[i, j]
        
        return predicted_mask
    
    def visualize_segmentation(self, img, ground_truth, predicted):
        """Visualiza resultados da segmentação"""
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Imagem original
        axes[0, 0].imshow(img)
        axes[0, 0].set_title('Imagem Original')
        axes[0, 0].axis('off')
        
        # Ground truth
        colors = ['black', 'lightblue', 'green', 'darkgreen', 'brown', 'darkred', 'white']
        cmap = ListedColormap(colors)
        
        im1 = axes[0, 1].imshow(ground_truth, cmap=cmap, vmin=0, vmax=6)
        axes[0, 1].set_title('Ground Truth')
        axes[0, 1].axis('off')
        
        # Predição
        im2 = axes[1, 0].imshow(predicted, cmap=cmap, vmin=0, vmax=6)
        axes[1, 0].set_title('Predição')
        axes[1, 0].axis('off')
        
        # Diferenças
        diff = np.abs(ground_truth.astype(float) - predicted.astype(float))
        im3 = axes[1, 1].imshow(diff, cmap='Reds', vmin=0, vmax=6)
        axes[1, 1].set_title('Diferenças (Vermelho = Erro)')
        axes[1, 1].axis('off')
        
        # Adicionar colorbar
        plt.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)
        plt.colorbar(im2, ax=axes[1, 0], fraction=0.046, pad=0.04)
        plt.colorbar(im3, ax=axes[1, 1], fraction=0.046, pad=0.04)
        
        plt.suptitle('Segmentação de Imagens', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return predicted
    
    def analyze_segmentation_metrics(self, ground_truth, predicted):
        """Analisa métricas de segmentação"""
        
        # Calcular métricas
        total_pixels = ground_truth.size
        correct_pixels = np.sum(ground_truth == predicted)
        pixel_accuracy = correct_pixels / total_pixels
        
        # Calcular IoU por classe
        ious = []
        class_names = ['Fundo', 'Céu', 'Chão', 'Árvore', 'Casa', 'Telhado', 'Nuvem']
        
        for class_id in range(7):
            gt_mask = (ground_truth == class_id)
            pred_mask = (predicted == class_id)
            
            intersection = np.sum(gt_mask & pred_mask)
            union = np.sum(gt_mask | pred_mask)
            
            iou = intersection / union if union > 0 else 0
            ious.append(iou)
        
        # Calcular mIoU
        mean_iou = np.mean(ious)
        
        # Visualizar métricas
        fig, axes = plt.subplots(1, 2, figsize=(18, 6))
        
        # Gráfico de IoU por classe
        axes[0].bar(class_names, ious, color=['black', 'lightblue', 'green', 'darkgreen', 'brown', 'darkred', 'white'])
        axes[0].set_title('IoU por Classe')
        axes[0].set_ylabel('IoU')
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(ious):
            axes[0].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        # Gráfico de métricas gerais
        metrics = ['Pixel Accuracy', 'Mean IoU']
        values = [pixel_accuracy, mean_iou]
        
        axes[1].bar(metrics, values, color=['lightblue', 'lightgreen'])
        axes[1].set_title('Métricas Gerais')
        axes[1].set_ylabel('Valor')
        axes[1].set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, v in enumerate(values):
            axes[1].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
        
        plt.suptitle('Métricas de Segmentação', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DAS MÉTRICAS ===")
        print(f"Pixel Accuracy: {pixel_accuracy:.2f}")
        print(f"Mean IoU: {mean_iou:.2f}")
        print(f"\nIoU por Classe:")
        for i, (name, iou) in enumerate(zip(class_names, ious)):
            print(f"  {name}: {iou:.2f}")
        
        return {
            'pixel_accuracy': pixel_accuracy,
            'mean_iou': mean_iou,
            'iou_per_class': dict(zip(class_names, ious))
        }

def demonstrate_image_segmentation():
    """Demonstra segmentação de imagens"""
    
    print("=== Demonstração de Segmentação de Imagens ===")
    
    # Criar instância
    demo = ImageSegmentationDemo()
    
    # Criar cena de exemplo
    print("\nCriando cena de exemplo...")
    img = demo.create_sample_scene()
    
    # Criar ground truth
    print("\nCriando ground truth...")
    ground_truth = demo.create_ground_truth_mask(img)
    
    # Simular segmentação
    print("\nSimulando segmentação...")
    predicted = demo.simulate_segmentation(img, ground_truth)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    demo.visualize_segmentation(img, ground_truth, predicted)
    
    # Analisar métricas
    print("\nAnalisando métricas...")
    metrics = demo.analyze_segmentation_metrics(ground_truth, predicted)
    
    return predicted, metrics

# Executar demonstração
segmentation_results, segmentation_metrics = demonstrate_image_segmentation()

### Análise dos Resultados

**Segmentação Observada:**

1. **Classes Identificadas**: 7 classes (Fundo, Céu, Chão, Árvore, Casa, Telhado, Nuvem)
2. **Pixel Accuracy**: ~95% de pixels corretos
3. **Mean IoU**: ~0.85 de interseção sobre união
4. **Erros**: Principalmente em bordas e transições

**Insights Importantes:**
- **Segmentação semântica**: Cada pixel classificado
- **Métricas**: Pixel accuracy e IoU
- **Desafios**: Bordas e transições
- **Aplicações**: Medicina, automotivo, agricultura

**Referências:**
- [U-Net: Convolutional Networks for Biomedical Image Segmentation - Ronneberger et al.](https://arxiv.org/abs/1505.04597)


## 5.7 Comparação das Tarefas### Diferenças Fundamentais**Classificação:**- **Entrada**: Imagem completa- **Saída**: Classe(es) da imagem- **Granularidade**: Nível de imagem- **Objetivo**: "O que" está na imagem**Detecção:**- **Entrada**: Imagem completa- **Saída**: Bounding boxes + classes- **Granularidade**: Nível de objeto- **Objetivo**: "O que" e "onde" estão os objetos**Segmentação:**- **Entrada**: Imagem completa- **Saída**: Mapa de classes por pixel- **Granularidade**: Nível de pixel- **Objetivo**: "O que" está em cada pixel![Comparação das Tarefas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/comparacao_tarefas.png)### Métricas de Avaliação**Classificação:**- **Accuracy**: Proporção de predições corretas- **Precision**: TP / (TP + FP)- **Recall**: TP / (TP + FN)- **F1-Score**: 2 × (Precision × Recall) / (Precision + Recall)**Detecção:**- **mAP**: Mean Average Precision- **IoU**: Intersection over Union- **Precision**: Detecções corretas / total de detecções- **Recall**: Detecções corretas / total de objetos**Segmentação:**- **Pixel Accuracy**: Pixels corretos / total de pixels- **Mean IoU**: Média do IoU por classe- **Dice Coefficient**: 2 × Intersection / (Ground Truth + Prediction)- **F1-Score**: Para cada classe![Métricas de Avaliação](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo5/metricas_avaliacao.png)**Referências:**- [Deep Learning for Computer Vision - Goodfellow, Bengio & Courville](https://www.deeplearningbook.org/)

## Resumo do Módulo 5

### Principais Conceitos Abordados

1. **Classificação de Imagens**
   - Definição e características
   - Arquiteturas específicas
   - Métricas de avaliação

2. **Detecção de Objetos**
   - Localização e classificação
   - Modelos clássicos (YOLO, R-CNN)
   - Métricas específicas

3. **Segmentação de Imagens**
   - Tipos de segmentação
   - Arquiteturas especializadas
   - Métricas de pixel

### Demonstrações Práticas

**1. Classificação:**
   - Implementação de classificador
   - Análise de probabilidades
   - Métricas de performance

**2. Detecção:**
   - Simulação de detecção
   - Visualização de bounding boxes
   - Análise de métricas

**3. Segmentação:**
   - Criação de máscaras
   - Visualização de segmentos
   - Análise de IoU

### Próximos Passos

No próximo módulo, exploraremos **OCR e Reconhecimento de Texto**, onde aprenderemos sobre extração e reconhecimento de texto em imagens.

### Referências Principais

- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)
- [You Only Look Once: Unified, Real-Time Object Detection - Redmon et al.](https://arxiv.org/abs/1506.02640)
- [U-Net: Convolutional Networks for Biomedical Image Segmentation - Ronneberger et al.](https://arxiv.org/abs/1505.04597)

---

**Próximo Módulo**: OCR e Reconhecimento de Texto
